### Dataset

In [ ]:
from predict360user import Dataset, Plot360

ds = Dataset(dataset_name="all")
display(ds.df.head())
display(ds.df["actS_c"].value_counts())

In [ ]:
ds.show_entropy_histogram()

### Plot360 a trace

In [ ]:
# show tile covering for cardinal points
look_north_pole = [0.0, 0.0, 1.0]
look_south_pole = [0.0, 0.0, -1.0]
look_left = [0.0, -1.0, 0.0]
look_right = [0.0, 1.0, 0.0]
look_ahead = [1.0, 0.0, 0.0]
look_behind = [-1.0, 0.0, 0.0]
Plot360().show_fov(look_north_pole)
Plot360().show_fov(look_south_pole)
Plot360().show_fov(look_left)
Plot360().show_fov(look_right)
Plot360().show_fov(look_ahead)
Plot360().show_fov(look_behind)


### Plot360 a trajec

In [ ]:
# show random traject 
ds.show_traject(ds.df.sample(1).iloc[0])

In [ ]:
# show one traject
ds.show_traject(ds.df.loc["david","0", "10_Cows"])

### Dataset.show_histogram

In [ ]:
ds.show_histogram()

In [ ]:
# show representive trajectory for each actS_c
Plot360().show_traject(ds.df[ds.df['actS_c'] == 'low'].nsmallest(1,'actS'))
Plot360().show_traject(ds.df[ds.df['actS_c'] == 'medium'].nsmallest(1,'actS'))
Plot360().show_traject(ds.df[ds.df['actS_c'] == 'high'].nlargest(1,'actS'))

### Dataset.calc_traces_poles_prc


In [ ]:
# calculate poles_prc_c
if not 'poles_prc_c' in ds.df.columns:
  ds.calc_traces_poles_prc()
ds.show_histogram(['poles_prc'])

In [ ]:
# show representive trajectory for each poles class
Plot360().show_traject(ds.df[ds.df['poles_prc_c'] == 'low'][:1])
Plot360().show_traject(ds.df[ds.df['poles_prc_c'] == 'medium'][:1])
Plot360().show_traject(ds.df[ds.df['poles_prc_c'] == 'high'][:1])